In [1]:
!pip install tiny-tokenizer
!pip install pytorch_transformers flair seqeval

  Created wheel for tiny-tokenizer: filename=tiny_tokenizer-3.1.0-cp36-none-any.whl size=10549 sha256=c51f28696da143498a99b3843e21bf07204279bd16c33a4269bb5aed9340bf9e
  Stored in directory: /root/.cache/pip/wheels/d1/c8/36/334497a689fab90128232e86b5829b800dd271a3d5d5959c53
Successfully built tiny-tokenizer
     |████████████████████████████████| 184kB 11.1MB/s 
     |████████████████████████████████| 143kB 32.5MB/s 
     |████████████████████████████████| 1.0MB 50.7MB/s 
     |████████████████████████████████| 890kB 52.6MB/s 
     |████████████████████████████████| 256kB 41.4MB/s 
     |████████████████████████████████| 983kB 52.8MB/s 
     |████████████████████████████████| 798kB 44.4MB/s 
     |████████████████████████████████| 573kB 51.0MB/s 
     |████████████████████████████████| 3.7MB 45.3MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=8389556ca8bb2b5bf16df293650e6a260e3cc3fc7e3859e70a176f62526de391
  Stored in directory: /root/.cache

In [0]:
# from bert_sequence_tagger.metrics import classification_report


In [0]:
# %load_ext autoreload
#%autoreload 2


import torch
from flair.datasets import ColumnCorpus
from pytorch_transformers import BertTokenizer, BertForTokenClassification
import torch.nn as nn
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pytorch_transformers import AdamW, WarmupLinearSchedule
from tqdm import trange
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

import os
import sys

In [0]:
device = torch.device('cuda')
# n_gpu = torch.cuda.device_count()

# for i in range(n_gpu):
#     print(torch.cuda.get_device_name(i))

In [0]:
MAX_LEN = 80
# CACHE_DIR = '../workdir/cache'
CACHE_DIR = 'cache/'

In [5]:
!unzip seq_lab_data.zip

Archive:  seq_lab_data.zip
   creating: manual_testset/
  inflating: manual_testset/test_manual_aspect_seq_labelling.conll  
  inflating: manual_testset/test_manual_clf_binary.csv  
  inflating: manual_testset/test_manual_predfull_seq_labelling.conll  
  inflating: manual_testset/test_manual_clf_fine_grained.csv  
   creating: sequence_labelling_aspect/
  inflating: sequence_labelling_aspect/dev_aspect.tsv  
  inflating: sequence_labelling_aspect/test_aspect.tsv  
  inflating: sequence_labelling_aspect/train_aspect.tsv  
  inflating: sequence_labelling_aspect/dev_aspect.xlsx  
   creating: sequence_labelling_predicate/
  inflating: sequence_labelling_predicate/dev_pred_full.tsv  
  inflating: sequence_labelling_predicate/test_pred_full.tsv  
  inflating: sequence_labelling_predicate/train_pred_full.tsv  


In [4]:
data = "pred"

if data == "asp":
  data_folder = 'sequence_labelling_aspect'
  corpus = ColumnCorpus(data_folder, 
                        {0 : 'text', 1 : 'ner'},
                        train_file='train_aspect.tsv',
                        test_file='test_aspect.tsv',
                        dev_file='dev_aspect.tsv')
elif data == "pred":
  data_folder = 'sequence_labelling_predicate'
  corpus = ColumnCorpus(data_folder, 
                        {0 : 'text', 1 : 'ner'},
                        train_file='train_pred_full.tsv',
                        test_file='test_pred_full.tsv',
                        dev_file='dev_pred_full.tsv')

print(corpus.obtain_statistics())

2020-04-20 21:00:17,395 Reading data from sequence_labelling_predicate
2020-04-20 21:00:17,397 Train: sequence_labelling_predicate/train_pred_full.tsv
2020-04-20 21:00:17,398 Dev: sequence_labelling_predicate/dev_pred_full.tsv
2020-04-20 21:00:17,399 Test: sequence_labelling_predicate/test_pred_full.tsv
{
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 3077,
        "number_of_documents_per_class": {},
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 89350,
            "min": 6,
            "max": 106,
            "avg": 29.038024049398764
        }
    },
    "TEST": {
        "dataset": "TEST",
        "total_number_of_documents": 488,
        "number_of_documents_per_class": {},
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 13736,
            "min": 6,
            "max": 97,
            "avg": 28.147540983606557
        }
    },
    "DEV": {
        "dataset": "DEV"

In [5]:
lengths = []

for sent in corpus.train:
  lengths.append(len(sent.tokens) + 2)

len_ser = pd.Series(lengths)
len_ser.quantile([.95, .96, .97, .98, .99, 1.0])

0.95     52.00
0.96     53.00
0.97     56.00
0.98     60.00
0.99     73.24
1.00    108.00
dtype: float64

In [0]:
def make_bert_tag_dict_from_flair_corpus(corpus):
    tags_vals = corpus.make_tag_dictionary('ner').get_items()
    tags_vals.remove('<unk>')
    tags_vals.remove('<START>')
    tags_vals.remove('<STOP>')
    tags_vals = ['<pad>'] + tags_vals
    tag2idx = {t : i for i, t in enumerate(tags_vals)}
    return tags_vals, tag2idx


BPE_TOKENIZER = BertTokenizer.from_pretrained('bert-base-cased', cache_dir=CACHE_DIR, do_lower_case=False)

IDX2TAG, TAG2IDX = make_bert_tag_dict_from_flair_corpus(corpus)

model = BertForTokenClassification.from_pretrained('bert-base-cased', cache_dir=CACHE_DIR, num_labels=len(TAG2IDX)).cuda()
# model = BertForTokenClassification.from_pretrained(MODEL_DIR, cache_dir=CACHE_DIR, num_labels=len(TAG2IDX)).cuda()

In [7]:
TAG2IDX

{'<pad>': 0, 'B-OBJ': 2, 'B-PREDFULL': 3, 'I-OBJ': 5, 'I-PREDFULL': 4, 'O': 1}

In [0]:
def prepare_flair_corpus(corpus, name='ner', filter_tokens={'-DOCSTART-'}):
    result = []
    for sent in corpus:
        if sent[0].text in filter_tokens:
            continue
        else:
            result.append(([token.text for token in sent.tokens],
                           [token.tags[name].value for token in sent.tokens]))
    
    return result


train_dataset = prepare_flair_corpus(corpus.train)
dev_dataset = prepare_flair_corpus(corpus.dev)
test_dataset = prepare_flair_corpus(corpus.test)

In [0]:
def bpe_tokenize(bpe_tokenizer, words):
    new_words = []
    bpe_masks = []
    for word in words:
        bpe_tokens = bpe_tokenizer.tokenize(word)
        new_words += bpe_tokens        
        bpe_masks += [1] + [0] * (len(bpe_tokens) - 1)

    return new_words, bpe_masks

In [0]:
def add_labels_for_bpe_suffix(labels, bpe_masks):
    result_labels = []
    for l_sent, m_sent in zip(labels, bpe_masks):
        m_sent = m_sent
        sent_res = []
        i = 0
        for l in l_sent:
            sent_res.append(l)

            i += 1
            while i < len(m_sent) and (m_sent[i] == 0):
                i += 1
                sent_res.append('<pad>')

        result_labels.append(sent_res)

    return result_labels

In [0]:
def prepare_bpe_tokens_for_bert(tokens, max_len):
    return [['[CLS]'] + list(toks[:max_len - 2]) + ['[SEP]'] for toks in tokens]


def prepare_bpe_labels_for_bert(labels, max_len):
    return [['<pad>'] + list(ls[:max_len - 2]) + ['<pad>'] for ls in labels]

In [0]:
def generate_masks(input_ids):
    return input_ids > 0


def make_tokens_tensors(tokens, max_len):
    bpe_tokens, bpe_masks = tuple(zip(*[bpe_tokenize(BPE_TOKENIZER, sent) for sent in tokens]))
    bpe_tokens = prepare_bpe_tokens_for_bert(bpe_tokens, max_len=max_len)
    bpe_masks = [masks[:max_len-2] for masks in bpe_masks]
    max_len = max(len(sent) for sent in bpe_tokens)
    token_ids = torch.tensor(pad_sequences([BPE_TOKENIZER.convert_tokens_to_ids(sent) for sent in bpe_tokens], 
                                           maxlen=max_len, dtype='long', 
                                           truncating='post', padding='post'))
    token_masks = generate_masks(token_ids)
    return bpe_tokens, max_len, token_ids, token_masks, bpe_masks
    
    
def make_label_tensors(labels, bpe_masks, max_len):
    bpe_labels = add_labels_for_bpe_suffix(labels, bpe_masks)
    bpe_labels = prepare_bpe_labels_for_bert(bpe_labels, max_len=max_len)
    
    label_ids = pad_sequences([[TAG2IDX.get(l) for l in lab] for lab in bpe_labels],
                         maxlen=max_len, value=TAG2IDX['<pad>'], padding='post',
                         dtype='long', truncating='post')
    return torch.LongTensor(label_ids)

In [0]:
def generate_tensors_for_training(tokens, labels):
    _, max_len, token_ids, token_masks, bpe_masks = make_tokens_tensors(tokens, MAX_LEN)
    label_ids = make_label_tensors(labels, bpe_masks, max_len)
    return token_ids, token_masks, label_ids


def batch_loss_tensors(bert_model, *tensors):
    token_ids, token_masks, label_ids = tensors
    token_ids = token_ids.cuda()
    token_masks = token_masks.cuda()
    label_ids = label_ids.cuda()

    output = bert_model(token_ids, token_type_ids=None, 
                        attention_mask=token_masks, 
                        labels=label_ids)

    loss = output[0]
    return loss.mean()

In [0]:
BATCH_SIZE = 24
MAX_N_EPOCHS = 500
WEIGHT_DECAY = 0.01
LEARNING_RATE = 1e-6
PRED_BATCH_SIZE = 1

optimizer = AdamW(model.parameters(), 
                  lr=LEARNING_RATE, betas=(0.9, 0.999), 
                  eps =1e-6, weight_decay=0.01, correct_bias=True)
lr_scheduler = WarmupLinearSchedule(optimizer, warmup_steps=0.1, 
                                    t_total=(len(train_dataset) / BATCH_SIZE)*MAX_N_EPOCHS)

In [0]:
def make_tensors(dataset_row):
    tokens, labels = tuple(zip(*dataset_row))
    return generate_tensors_for_training(tokens, labels)
    
    
def train(model, optimizer, lr_scheduler, train_dataset):
    train_dataloader = DataLoader(train_dataset, 
                                  batch_size=BATCH_SIZE, 
                                  shuffle=True,
                                  collate_fn=make_tensors, 
                                  num_workers=1)
    iterator = trange(MAX_N_EPOCHS, desc='Epoch')
    for epoch in iterator:
        model.train()

        cum_loss = 0.
        for nb, tensors in enumerate(train_dataloader):
            loss = batch_loss_tensors(model, *tensors)
            cum_loss += loss.item()

            model.zero_grad()
    
            loss.backward()
        
            torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), 
                                           max_norm=1.)

            optimizer.step()
            lr_scheduler.step()

        cum_loss /= (nb + 1)
        print(f'Train loss: {cum_loss}')

In [44]:
train(model, optimizer, lr_scheduler, train_dataset)

Epoch:   0%|          | 1/500 [00:54<7:30:34, 54.18s/it]

Train loss: 1.207322518492854


Epoch:   0%|          | 2/500 [01:48<7:30:04, 54.23s/it]

Train loss: 0.6011341976564984


Epoch:   1%|          | 3/500 [02:43<7:30:12, 54.35s/it]

...

Train loss: 0.00034708114899401624


Epoch: 100%|█████████▉| 499/500 [7:33:48<00:54, 54.43s/it]

Train loss: 0.0002767575808193638


Epoch: 100%|██████████| 500/500 [7:34:43<00:00, 54.57s/it]

Train loss: 0.00021976542000827116


In [0]:
def generate_tensors_for_prediction(evaluate, dataset_row):
    dataset_row = dataset_row
    labels = None
    if evaluate:
        tokens, labels = tuple(zip(*dataset_row))
    else:
        tokens = dataset_row

    _, max_len, token_ids, token_masks, bpe_masks = make_tokens_tensors(tokens, MAX_LEN)
    label_ids = None

    if evaluate:
        label_ids = make_label_tensors(labels, bpe_masks, max_len)

    return token_ids, token_masks, bpe_masks, label_ids, tokens, labels
    

def logits_to_preds(logits, bpe_masks, tokens):
    preds = logits.argmax(dim=2).numpy()
    probs = logits.numpy().max(axis=2)
    prob = [np.mean([p for p, m in zip(prob[1:-1][:len(masks)], masks[:len(prob)-2]) if m])  
            for prob, masks in zip(probs, bpe_masks)]
    preds = [[IDX2TAG[p] for p, m in zip(pred[1:-1][:len(masks)], masks[:len(pred)-2]) if m] 
             for pred, masks in zip(preds, bpe_masks)]
    preds = [pred + ['O']*(max(0, len(toks) - len(pred))) for pred, toks in zip(preds, tokens)]
    return preds, prob


def predict(bert_model, dataset, evaluate=False, metrics=None, pred_loader_args={'num_workers' : 1}):
    if metrics is None:
        metrics = []

    bert_model.eval()

    dataloader = DataLoader(dataset, 
                            collate_fn=lambda dataset_row: generate_tensors_for_prediction(evaluate, dataset_row), 
                            **pred_loader_args, 
                            batch_size=PRED_BATCH_SIZE)

    predictions = []
    probas = []

    if evaluate:
        cum_loss = 0.
        true_labels = []

    for nb, tensors in enumerate(dataloader):
        token_ids, token_masks, bpe_masks, label_ids, tokens, labels = tensors

        if evaluate:
            true_labels.extend(labels)

        with torch.no_grad():
            token_ids = token_ids.cuda()
            token_masks = token_masks.cuda()

            if evaluate:
                label_ids = label_ids.cuda()

            logits = bert_model(token_ids, 
                                token_type_ids=None,
                                attention_mask=token_masks,
                                labels=label_ids,)

            if evaluate:
                loss, logits = logits
                cum_loss += loss.mean().item()
            else:
                logits = logits[0]

            b_preds, b_prob = logits_to_preds(logits.cpu(), bpe_masks, tokens)

        predictions.extend(b_preds)
        probas.extend(b_prob)

    if evaluate: 
        cum_loss /= (nb + 1)

        result_metrics = []
        for metric in metrics:
            result_metrics.append(metric(true_labels, predictions))

        return predictions, probas, tuple([cum_loss] + result_metrics)
    else:
        return predictions, probas

In [46]:
_, __, test_metrics = predict(model, test_dataset, evaluate=True, metrics=[precision_score, recall_score, f1_score, classification_report])
print(f'Loss:', test_metrics[0])
print(f'Precision:', test_metrics[1])
print(f'Recall:', test_metrics[2])
print(f'Entity-level f1: {str(test_metrics[3])}')
print(f'Report:\n {str(test_metrics[4])}')

Loss: 0.4684708160984342
Precision: 0.6993087557603687
Recall: 0.48328025477707004
Entity-level f1: 0.5715630885122411
Report:
            precision    recall  f1-score   support

      OBJ       0.74      0.46      0.57      1092
 PREDFULL       0.58      0.61      0.60       164

micro avg       0.70      0.48      0.57      1256
macro avg       0.72      0.48      0.57      1256



In [0]:
MODEL_DIR = "model/"
CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

In [0]:
if not os.path.exists(MODEL_DIR):
  os.makedirs(MODEL_DIR)

In [0]:
model_to_save = model.module if hasattr(model, 'module') else model

output_model_file = os.path.join(MODEL_DIR, WEIGHTS_NAME)
output_config_file = os.path.join(MODEL_DIR, CONFIG_NAME)

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)